In [18]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
import copy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [2]:
client_id = open('client.txt').readlines()[0]
secret = open('secret.txt').readlines()[0]
username = open('username.txt').readlines()[0]
redirect_url = open('redirect.txt').readlines()[0]

'https://github.com/Michael-Faulkner/Spotify_Playlist'

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=secret) 
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
scope = 'user-library-read playlist-read-private user-top-read playlist-modify-public'
token = util.prompt_for_user_token(username, scope,client_id, secret, redirect_url)
if token:
    spotify = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [5]:
results = spotify.current_user_top_tracks(limit = 50, time_range = 'long_term')
x = pd.DataFrame(results['items'])
working = copy.deepcopy(x)
for i in range(len(x)):
    
    artist_list = []
    if len(x.loc[i,'artists']) > 1:
        for artist in x.loc[i, 'artists']:
            artist_list.append(artist['name'])
        working.at[i,'artists'] = artist_list
    else:
        working.at[i, 'artists'] = x.loc[i, 'artists'][0]['name']

    working.at[i,'album'] = x.loc[i,'album']['name']
    
working = working.drop(['available_markets', 'disc_number', 'explicit', 'external_ids', 'external_urls', 'href','preview_url', 'track_number', 'type', 'uri','is_local','duration_ms' ], axis = 1)
working

,album,artists,id,name,popularity
0,Vagrant,"[Feint, Veela]",1M85o2ruB9uTj68dIXj6dJ,Vagrant,54
1,UKF Drum & Bass 2016,Sigma,23L2jzpnyfxbCmliMBMGQM,Stay - Maduk Remix,40
2,Getaway (Remixes),"[Tritonal, Angel Taylor, Koven]",10EpXLXKHmNSVKvX7A5hg8,Getaway - Koven Remix,54
3,Into You (3LAU Remix),"[Ariana Grande, 3LAU]",1cfVkL84XmRON3Gl9jDXKI,Into You - 3LAU Remix,53
4,Feels (Jai Wolf Remix),"[Kiiara, Jai Wolf]",57rOXabUKgkrwHPSqzo4s5,Feels - Jai Wolf Remix,54
5,Ain't My Fault (R3hab Remix),"[Zara Larsson, R3HAB]",3YPlsrk0Voe8AOa1jyN891,Ain't My Fault - R3hab Remix,60
6,Creed II: The Album,"[Mike WiLL Made-It, A$AP Rocky, A$AP Ferg, Nic...",23dwrzeJtEEGl2U4CPWSor,"Runnin (with A$AP Rocky, A$AP Ferg & Nicki Minaj)",65
7,Close,"[IZECOLD, Molly Ann]",454IPfzFpYJsCvqbS0UDbR,Close,43
8,Crazy Love (Niko The Kid Remix),"[Audien, Deb’s Daughter, Niko The Kid]",7tdBWDVM5W5Bkx0mXsnYzW,Crazy Love - Niko The Kid Remix,23
9,Friends,"[It's Different, Drama B]",2FXreiGteQpcRuMuvaSQeX,Friends,23


In [6]:
y = spotify.current_user_playlists()['items']
playlists = pd.DataFrame(y)
playlists = playlists[['name','id']]


good_playlist_id = playlists[playlists['name'] == 'Python Test 1']
good_playlist_id = good_playlist_id.iloc[0,1]


bad_playlist_id = playlists[playlists['name'] == 'Python Test 0']
bad_playlist_id = bad_playlist_id.iloc[0,1]

In [7]:
good_uncleaned = []
for i in range(10):
    try:
        good_uncleaned.append(pd.DataFrame(spotify.playlist_tracks(good_playlist_id, offset = 100 * i)['items'])['track'])
    except:
        break
good_uncleaned_tracks = pd.concat(good_uncleaned)
good_uncleaned_tracks = good_uncleaned_tracks.reset_index(drop = True)
good_track_ids = []

for i in range(len(good_uncleaned_tracks)):
    song_id = good_uncleaned_tracks[i]['id']
    good_track_ids.append(song_id)
good_feats = []
    
for i in range(10):
 
    try:
        good_feats.append(pd.DataFrame(spotify.audio_features(good_track_ids[i*100:(i+1)*100])))
    except:
        print(i)
        good_feats.append(pd.DataFrame(spotify.audio_features(good_track_ids[i*100:])))
        break

good_audio_feats = pd.concat(good_feats)
good_audio_feats = good_audio_feats.drop(0, axis = 1)
good_audio_feats = good_audio_feats.reset_index(drop = True)
good_audio_feats = good_audio_feats.dropna()
good_audio_feats['target'] = 1

In [8]:
len(good_audio_feats)

345

In [9]:
bad_uncleaned  = []

for i in range(10):
    try:
        bad_uncleaned.append(pd.DataFrame(spotify.playlist_tracks(bad_playlist_id, offset = 100*i)['items'])['track'])
    except:
        break

bad_uncleaned_tracks = pd.concat(bad_uncleaned)

bad_uncleaned_tracks = bad_uncleaned_tracks.reset_index(drop = True)
bad_track_ids = []

for i in range(len(bad_uncleaned_tracks)):
    song_id = bad_uncleaned_tracks[i]['id']
    bad_track_ids.append(song_id)
bad_feats = []
for i in range(10):
 
    try:
        bad_feats.append(pd.DataFrame(spotify.audio_features(bad_track_ids[i*100:(i+1)*100])))
    except:
        print(i)
        bad_feats.append(pd.DataFrame(spotify.audio_features(bad_track_ids[i*100:])))
        break

bad_audio_feats = pd.concat(bad_feats)
bad_audio_feats = bad_audio_feats.drop(0, axis = 1)
bad_audio_feats = bad_audio_feats.reset_index(drop = True)
bad_audio_feats = bad_audio_feats.dropna()
bad_audio_feats['target'] = 0



In [10]:
all_audio_feats = pd.concat([good_audio_feats, bad_audio_feats])
all_audio_feats = all_audio_feats.reset_index(drop = True)
from sklearn.model_selection import train_test_split
train, test = train_test_split(all_audio_feats, test_size = 0.2)

In [14]:
features = ["danceability", "loudness", "valence", "energy", "instrumentalness", "acousticness", "key", "speechiness", 'liveness',"tempo"]

x_train = train[features]
y_train = train["target"]
x_test = test[features]
y_test = test["target"]

ss = StandardScaler()
x_train_scale = ss.fit_transform(x_train)
x_test_scale = ss.transform(x_test)

In [35]:
knn = KNeighborsClassifier(23, weights = 'uniform', metric = 'manhattan')
knn.fit(x_train_scale, y_train)
knn_pred = knn.predict(x_test_scale)
score = accuracy_score(y_test, knn_pred) * 100
print("Accuracy using Knn Tree: ", round(score, 1), "%")

Accuracy using Knn Tree:  78.2 %


In [33]:
grid_params = {'n_neighbors':[ 23],
              'weights':['uniform', 'distance'],
              'metric':['euclidean', 'manhattan']}
gs = GridSearchCV(KNeighborsClassifier(), grid_params)

gs_results = gs.fit(x_train_scale, y_train)

In [34]:
gs_results.best_params_

{'metric': 'manhattan', 'n_neighbors': 23, 'weights': 'uniform'}

In [ ]:
clf = RandomForestClassifier()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
score = accuracy_score(y_test, y_pred) * 100
print("Accuracy using Decision Tree: ", round(score, 1), "%")

In [ ]:
song_db = pd.read_csv('./data.csv')

In [ ]:
song_db_features = song_db[features]
a = list(knn.predict(song_db_features))
b = list(clf.predict(song_db_features))
song_db['result_rf'] = b
song_db['result_knn'] = a
x = song_db[features + ['result_rf','result_knn','id']]
good_songs = x[(x['result_rf'] == 1) & x['result_knn'] == 1]
len(good_songs)

In [ ]:
x = song_db[features + ['result_rf','result_knn','id']]
good_songs = x[(x['result_rf'] == 1) & x['result_knn'] == 1]
len(good_songs)


In [ ]:
spotify.track('764wHEOb2yPgsJHmU0J4ZS')['album']['artists'][0]['name']

In [26]:
len(y_train)

841

In [ ]:
tth = playlists[playlists['name'] == 'Today\'s Top Hits']
tth_id = tth.iloc[0,1]
tth_uncleaned_tracks = pd.DataFrame(spotify.playlist_tracks(tth_id)['items'])['track']
tth_track_ids = []
for i in range(len(tth_uncleaned_tracks)):
    song_id = tth_uncleaned_tracks[i]['id']
    tth_track_ids.append(song_id)
tth_audio_feats = pd.DataFrame(spotify.audio_features(tth_track_ids))
tth_test = tth_audio_feats[features]
a = list(knn.predict(tth_test))
tth_audio_feats['result'] = a

today = tth_audio_feats[features + ['result','id']]

track_names = spotify.tracks(list(today['id']))

names_list = []
print(len(track_names))

for i in range(len(today)):
    
    names_list.append(track_names['tracks'][i]['name'])


today['song name'] = names_list

In [ ]:
spotify.tracks(list(good_songs['id'])[0:50])

In [ ]:
name = 'ML'
spotify.user_playlist_create(username, name, public=True, collaborative=False, description='')
for item in spotify.current_user_playlists()['items']:
    if item['name'] == name:
        new_playlist_id = item['id']
new_playlist_id

In [ ]:
songs_to_add = list(good_songs[:100]['id'])
spotify.playlist_add_items(new_playlist_id, songs_to_add)

In [ ]:
spotify.playlist_add_items(new_playlist_id, songs_to_add)

In [ ]:
spotify.recommendation_genre_seeds()

In [ ]:
wanted_genres = ['edm','dance']

In [ ]:
x = spotify.recommendations(seed_artists=None, seed_genres=wanted_genres, seed_tracks=None, limit=100)

In [ ]:
test_ids = []
for i in range(100):
    test_ids.append(x['tracks'][i]['id'])
  

In [ ]:
test_df = pd.DataFrame(spotify.audio_features(test_ids))

In [ ]:
song_test_features = test_df[features]
a = list(knn.predict(song_test_features))
b = list(clf.predict(song_test_features))
test_df['result_rf'] = b
test_df['result_knn'] = a
y = test_df[features + ['result_rf','result_knn','id']]
good_songs = y[(y['result_rf'] == 1) | y['result_knn'] == 1]


In [ ]:
test_song_list = []
test_ids = []
for i in range(len(good_track_ids)):

    x = spotify.recommendations(seed_artists=None, seed_genres=None, seed_tracks=[good_track_ids[i]], limit=3)
    for j in range(len(x['tracks'])):
        s_id = x['tracks'][j]['id']
        test_ids.append(pd.DataFrame(spotify.audio_features(s_id)))

y = pd.concat(test_ids)
y = y.reset_index(drop = True)
z2 = y.drop_duplicates()
song_test_features = z2[features]
a = list(knn.predict(song_test_features))
b = list(clf.predict(song_test_features))
z2['result_rf'] = b
z2['result_knn'] = a
result = z2[features + ['result_rf','result_knn','id']]
good_songs = result[(result['result_rf'] == 1) & result['result_knn'] == 1]
test_song_list.append(good_songs)


In [ ]:
name = 'ML'
spotify.user_playlist_create(username, name, public=True, collaborative=False, description='')
for item in spotify.current_user_playlists()['items']:
    if item['name'] == name:
        new_playlist_id = item['id']
new_playlist_id

In [ ]:
range(int(np.ceil(len(good_songs)/100)))

In [ ]:
songs_by_100 = int(np.ceil(len(good_songs)/100))
for i in range(songs_by_100):
    if i < (songs_by_100 - 1):
        songs_to_add = list(good_songs['id'][i*100:100*(i+1)])
        spotify.playlist_add_items(new_playlist_id, songs_to_add)
    else:
        songs_to_add = list(good_songs['id'][i*100:])
        spotify.playlist_add_items(new_playlist_id, songs_to_add)

In [ ]:
z = pd.concat(test_ids)
z = z.reset_index(drop = True)

In [ ]:
z

In [ ]:
z2 = y.drop_duplicates()
len(z2)